In [1]:
using BlockArrays

In [2]:
b = BlockArray(rand(4, 4), [2,2], [2, 2])

2×2-blocked 4×4 BlockArray{Float64,2}:
 0.896502  0.784559   │  0.327262   0.325753
 0.735495  0.446902   │  0.368187   0.215745
 ─────────────────────┼─────────────────────
 0.313646  0.712828   │  0.745333   0.591883
 0.483306  0.0937295  │  0.0443354  0.742027

In [3]:
b[Block(1, 1)]

2×2 Array{Float64,2}:
 0.896502  0.784559
 0.735495  0.446902

In [4]:
b[Block(2, 2)]

2×2 Array{Float64,2}:
 0.745333   0.591883
 0.0443354  0.742027

In [106]:
include("BFFPSV18.jl")

add_time! (generic function with 2 methods)

## Motor

In [102]:
# ======================================
# Model: Motor (8 variables, 2 inputs)
# ======================================

using SparseArrays

# =====================
# Problem specification
# =====================
I = [1, 2, 2, 3, 3, 3, 3, 4, 5, 6, 6, 7, 7, 7, 7, 8]
J = [2, 3, 2, 1, 2, 3, 4, 1, 6, 7, 6, 5, 6, 7, 8, 5]
vals = [1, 8487.2, -1.0865, -2592.1, -21.119, -698.91, -141399.0, 1.0, 1.0,
       8487.2, -1.0865, -2592.1, -21.119, -698.91, -141399.0, 1.0]
A = sparse(I, J, vals)

# initial set
X0 = Hyperrectangle([0.00225, 0.0, 0.0, 0.0, 0.00125, 0.0, 0.0, 0.0],
                    [0.00025, 0.0, 0.0, 0.0, 0.00025, 0.0, 0.0, 0.0])

# input set
B = sparse([4, 8], [1, 2], [-1.0, -1.0])
U = Hyperrectangle([0.23, 0.3], [0.07, 0.1])

partition_1D = [[i] for i in 1:8]; # 1D blocks
partition_2D = [(2*i-1:2*i) for i in 1:4] # 2D blocks

4-element Array{UnitRange{Int64},1}:
 1:2
 3:4
 5:6
 7:8

In [94]:
F = exp(Matrix(A) * 0.001)

8×8 Array{Float64,2}:
  0.996878      0.000973576  0.00335549  …  0.0            0.0     
 -8.86569       0.924956     5.91775        0.0            0.0     
 -1.8644       -0.0157699    0.438394       0.0            0.0     
  0.000999194   4.9316e-7    1.18764e-6     0.0            0.0     
  0.0           0.0          0.0            0.00335549    -0.167932
  0.0           0.0          0.0         …  5.91775     -474.463   
  0.0           0.0          0.0            0.438394     -98.652   
  0.0           0.0          0.0            1.18764e-6     0.999957

In [66]:
Ablk = BlockArray(Matrix(A), [2, 2, 2, 2], [2, 2, 2, 2])  # 2D blocks
#Ablk = BlockArray(Matrix(A), fill(1, 8), fill(1, 8)) # 1D blocks

4×4-blocked 8×8 BlockArray{Float64,2}:
     0.0    1.0     │     0.0         0.0  │      0.0    0.0     │     0.0         0.0
     0.0   -1.0865  │  8487.2         0.0  │      0.0    0.0     │     0.0         0.0
 ───────────────────┼──────────────────────┼─────────────────────┼────────────────────
 -2592.1  -21.119   │  -698.91  -141399.0  │      0.0    0.0     │     0.0         0.0
     1.0    0.0     │     0.0         0.0  │      0.0    0.0     │     0.0         0.0
 ───────────────────┼──────────────────────┼─────────────────────┼────────────────────
     0.0    0.0     │     0.0         0.0  │      0.0    1.0     │     0.0         0.0
     0.0    0.0     │     0.0         0.0  │      0.0   -1.0865  │  8487.2         0.0
 ───────────────────┼──────────────────────┼─────────────────────┼────────────────────
     0.0    0.0     │     0.0         0.0  │  -2592.1  -21.119   │  -698.91  -141399.0
     0.0    0.0     │     0.0         0.0  │      1.0    0.0     │     0.0         0.0

In [26]:
exp(Matrix(A) * 0.001)[1:2, :]

2×8 Array{Float64,2}:
  0.996878  0.000973576  0.00335549    -0.167932  0.0  0.0  0.0  0.0
 -8.86569   0.924956     5.91775     -474.463     0.0  0.0  0.0  0.0

In [103]:
#S = ConstrainedLinearControlContinuousSystem(Ablk, B, nothing, U)
S = ConstrainedLinearControlContinuousSystem(A, B, nothing, U)

P = InitialValueProblem(S, X0);

In [104]:
# UNIFORM 1D partition
opts = Options(:T => 20.0,
               :δ => 0.001,
               :N => 20_000,  # el N se puede sacar de T y de delta
               :vars=>[5],
               :partition => partition_2D,
               :set_type => Hyperrectangle{Float64, Vector{Float64}, Vector{Float64}},          
     #:set_type => Interval{Float64, IA.Interval{Float64}},
               :row_blocks => [[1,2], [3,4], [5,6], [7,8]],
               :column_blocks => [[1,2], [3,4], [5,6], [7,8]],
               :num_type => Float64,
               :block_indices => [1, 2, 3, 4]);
# varias de las opciones anteriores se pueden deducir de otras!

In [17]:
using BenchmarkTools

In [18]:
@btime sol = solve_BFFPSV18($P, $opts); # with MMatrix

  60.401 ms (320483 allocations: 21.40 MiB)


In [48]:
@btime sol = solve_BFFPSV18($P, $opts); # with Ablk, 1D blocks

  66.339 ms (342584 allocations: 22.44 MiB)


In [109]:
@btime sol = solve_BFFPSV18($P, $opts); # with Ablk, 2D blocks

  1.416 s (4699869 allocations: 329.62 MiB)


In [ ]:
# modified algorithm: we access Block(i)

In [107]:
sol = solve_BFFPSV18(P, opts);

In [70]:
Ablk

4×4-blocked 8×8 BlockArray{Float64,2}:
     0.0    1.0     │     0.0         0.0  │      0.0    0.0     │     0.0         0.0
     0.0   -1.0865  │  8487.2         0.0  │      0.0    0.0     │     0.0         0.0
 ───────────────────┼──────────────────────┼─────────────────────┼────────────────────
 -2592.1  -21.119   │  -698.91  -141399.0  │      0.0    0.0     │     0.0         0.0
     1.0    0.0     │     0.0         0.0  │      0.0    0.0     │     0.0         0.0
 ───────────────────┼──────────────────────┼─────────────────────┼────────────────────
     0.0    0.0     │     0.0         0.0  │      0.0    1.0     │     0.0         0.0
     0.0    0.0     │     0.0         0.0  │      0.0   -1.0865  │  8487.2         0.0
 ───────────────────┼──────────────────────┼─────────────────────┼────────────────────
     0.0    0.0     │     0.0         0.0  │  -2592.1  -21.119   │  -698.91  -141399.0
     0.0    0.0     │     0.0         0.0  │      1.0    0.0     │     0.0         0.0

In [71]:
include("BFFPSV18.jl")

2×2 Array{Float64,2}:
 0.0   1.0   
 0.0  -1.0865